In [ ]:
# Librerías a utilizar
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# Base URL sin número de página
base_url = 'https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_{}.html'

# Lista para almacenar los datos de todas las páginas
all_data = []

# Loop para iterar por las páginas 1 a 10
for page in range(1, 11):
    # Construir la URL para cada página
    url = base_url.format(page)
    
    # Enviar una solicitud HTTP
    response = requests.get(url)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Parsear el contenido con BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table')
        
        # Extraer encabezados solo en la primera iteración (ya se verifico que todas las paginas tienen el mismo encabezado)
        if page == 1:
            headers = [header.text.strip() for header in table.find_all('th')]
        
        # Extraer las filas de la tabla
        for row in table.find_all('tr'):
            row_data = [cell.text.strip() for cell in row.find_all('td')]
            if row_data:
                all_data.append(row_data)
    else:
        print(f"No se pudo acceder a la página {page}")

# Crear un DataFrame con los datos recopilados
df = pd.DataFrame(all_data, columns=headers)

# Mostrar las primeras filas del DataFrame
print(df.head())

In [ ]:
import os
import pdfplumber

In [ ]:
#ETIQUETAS DE VARIABLES 
# Ruta del archivo PDF local
pdf_path = r"ddi-documentation-spanish-608.pdf"

# Función para extraer tablas de las páginas 23 a 56 y convertirlas en un solo DataFrame o lista de DataFrames
def extract_tables_to_dataframe(pdf_path, start_page, end_page, combine=False):
    all_tables = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for i in range(start_page - 1, end_page):  # start_page=23, end_page=56
            page = pdf.pages[i]
            # Extraer las tablas de la página
            tables = page.extract_tables()
            
            # Convertir cada tabla a un DataFrame y agregarla a la lista
            for table in tables:
                df = pd.DataFrame(table[1:], columns=table[0])  # La primera fila es usada como cabecera
                all_tables.append(df)
    
    # Retorna un solo DataFrame concatenado si combine=True
    if combine:
        combined_df = pd.concat(all_tables, ignore_index=True)
        return combined_df
    else:
        return all_tables

# Extraer y combinar las tablas en un solo DataFrame de las páginas 23 a 56
combined_dataframe = extract_tables_to_dataframe(pdf_path, 23, 56, combine=True)
combined_dataframe.columns = list(range(1, len(combined_dataframe.columns) + 1))

combined_dataframe = combined_dataframe.drop([1, 2, 9], axis=1)
combined_dataframe = combined_dataframe.drop(index=list(range(0, 6)) + list(range(31, 37)))
combined_dataframe.columns = ["ID", "nombre", "Etiqueta", "Tipo", "Formato", "Pregunta"]
combined_dataframe.to_excel('descripcion variables.xlsx', index=False)

